## 1. 自定义块

In [8]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2660,  0.0450,  0.1234, -0.0009, -0.0774, -0.0315,  0.0144,  0.1165,
          0.0174,  0.2231],
        [ 0.2988,  0.0474,  0.0389,  0.0541,  0.0194, -0.0828, -0.0047,  0.1424,
          0.0642,  0.1637]], grad_fn=<AddmmBackward0>)

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层
        
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [10]:
'''实例化多层感知机'''
net = MLP() # 存在默认初始化方法
net(X)

tensor([[ 0.0730,  0.1179,  0.1814,  0.0862,  0.1326, -0.2338, -0.2105,  0.0573,
          0.1913, -0.1344],
        [ 0.0909,  0.0811,  0.1202,  0.0412,  0.2293, -0.2553, -0.1819,  0.0702,
          0.1990, -0.2521]], grad_fn=<AddmmBackward0>)

## 2. 顺序块

In [11]:
'''MySequential类提供了与默认Sequential类相同的功能'''
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_modules的类型是OrdereDict
            self._modules[str(idx)] = module
            
    def forward(self, X):
        # 由于_modules的类型是有序字典，所以可以有序便利其中的元素
        for block in self._modules.values():
            X = block(X)
        return X

In [12]:
'''使用自定义的有序块实现MLP'''
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1861,  0.1401,  0.0224,  0.0664,  0.0032, -0.3142, -0.0633, -0.1514,
         -0.1202,  0.0068],
        [-0.1935,  0.0154,  0.0150,  0.1118,  0.0492, -0.4230, -0.1266, -0.1531,
         -0.1402,  0.1289]], grad_fn=<AddmmBackward0>)

## 3. foward函数中执行代码

In [15]:
'''例如需要计算一个优化过程中没有更新的变量与参数和输入的运算结果'''
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。相当于两个全连接层共享参数
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
        

In [16]:
net = FixedHiddenMLP()
net(X) # 返回的就是forward函数中的结果

tensor(-0.1229, grad_fn=<SumBackward0>)

In [17]:
'''可以混搭各种组合块的方法'''
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
chimera(X)

tensor(0.1990, grad_fn=<SumBackward0>)